# LangChain

## LangSmith 추적 설정

- LLM 앱 개발, 모니터링 및 테스트를 위한 플랫폼
- https://smith.langchain.com/ 접속하여 회원가입/로그인 진행

- 생성한 API Key를 넣고 설정

```.env
LANGSMITH_TRACING=true
LANGSMITH_ENDPOINT=https://api.smith.langchain.com
LANGSMITH_API_KEY=API 키
LANGSMITH_PROJECT=프로젝트명
```

In [ ]:
%pip install python-dotenv

In [1]:
from dotenv import load_dotenv
# OPEN AI API 키 설정
load_dotenv()

True

In [ ]:
%pip install -U langchain langchain-openai

In [3]:
import os

def project_setup(name):
    os.environ['LANGSMITH_PROJECT'] = name
    os.environ['LANGSMITH_TRACING'] = 'true'
    print('프로젝트: {} 추적 시작'.format(name))

def stop_tracing():
    os.environ['LANGSMITH_TRACING'] = 'false'
    print('LangSmith 추적 중지')

project_setup('LangChain Test')

프로젝트: LangChain Test 추적 시작


### Open AI가 제공하는 모델

- https://platform.openai.com/docs/models

In [4]:
OPENAI_MODELS = {
    "gpt-4.1": {
        'price': {
            'input': 2.00, # per 1M tokens
            'cached': 0.5,
            'output': 8.00,
        }
    },
    "gpt-4.1-mini": {
        'price': {
            'input': 0.40,
            'cached': 0.1,
            'output': 1.60,
        }
    },
    "gpt-4.1-nano": { # 가장 저렴한 모델
        'price': {
            'input': 0.10,
            'cached': 0.025,
            'output': 0.40,
        }
    },
    "gpt-4o": {
        'price': {
            'input': 2.50,
            'cached': 1.25,
            'output': 10.00,
        }
    },
    "gpt-4o-mini": { 
        'price': {
            'input': 0.15,
            'cached': 0.075,
            'output': 0.60,
        }
    },
    "o3": {
        'price': {
            'input': 10.00,
            'cached': 2.50,
            'output': 40.00,
        }
    },
    # Usage Tier를 올려야 사용 가능
    "o4-mini": { # 가장 최신 모델 25-04-16
        'price': {
            'input': 1.10,
            'cached': 0.275,
            'output': 4.40,
        }
    },
    # 예전 모델
    "gpt-3.5-turbo": {
        'price': {
            'input': 0.5,
            'output': 1.50,
        }
    }
}

MODEL_NAMES = list(OPENAI_MODELS.keys())

In [5]:
MODEL_NAMES

['gpt-4.1',
 'gpt-4.1-mini',
 'gpt-4.1-nano',
 'gpt-4o',
 'gpt-4o-mini',
 'o3',
 'o4-mini',
 'gpt-3.5-turbo']

### LangChain 기초

In [6]:
from langchain_openai import ChatOpenAI

model_name = "gpt-4o-mini"

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048, # 최대 토큰수
    model_name=model_name,  # 모델명
) #.bind(logprobs=True)

In [ ]:
# 질문
question = "대한민국의 수도는 어디인가요?"

# 답변
answer = llm.invoke(question)
print(f"[답변]: {answer.content}") # 내용만
#print(f"[답변]: {answer}")

[답변]: 대한민국의 수도는 서울입니다.


In [9]:
type(answer)

langchain_core.messages.ai.AIMessage

In [10]:
# Stream 방식
question = "대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!"
answer = llm.stream(question)

for token in answer:
    print(token.content, end="", flush=True)

대한민국에는 아름다운 관광지가 많이 있습니다. 아래는 추천하는 10곳과 그 주소입니다.

1. **경복궁**
   - 주소: 서울특별시 종로구 사직로 161

2. **제주도**
   - 주소: 제주특별자치도 제주시

3. **부산 해운대 해수욕장**
   - 주소: 부산광역시 해운대구 해운대해변로 140

4. **경주 불국사**
   - 주소: 경상북도 경주시 불국로 385

5. **남이섬**
   - 주소: 강원도 춘천시 남이섬길 1

6. **전주 한옥마을**
   - 주소: 전라북도 전주시 완산구 기린대로 99

7. **설악산 국립공원**
   - 주소: 강원도 속초시 설악산로 173

8. **안동 하회마을**
   - 주소: 경상북도 안동시 풍천면 하회리

9. **서울 남산타워 (N서울타워)**
   - 주소: 서울특별시 용산구 남산공원길 105

10. **광주 무등산**
    - 주소: 광주광역시 동구 무등산로 100

각 관광지는 고유의 매력을 가지고 있으니, 방문 시 다양한 경험을 즐기시길 바랍니다!

## Prompt Engineering

### 프롬프트

- 생성형 모델의 출력을 원하는 방향으로 유도하기 위한 입력 텍스트


> #### Role
> System Prompt : 사용자의 prompt 이전에 **성능 개선 용도**의 프롬프트로 *페르소나 부여*나 응답 어조 등 **중요 규칙**을 설정 (system)  
> User Prompt : 사용자의 입력  (human)  
> Assitant: AI 모델을 의미함 (ai)  

#### ChatPromptTemplate

- 대화목록을 프롬프트로 주입하고자 할 때 활용


In [11]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages( # 별도의 템플릿으로 관리가능
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

In [ ]:
# 이런 형태는 자주 쓰진 않음
messages = template.format_messages(
    name="제이슨", user_input="당신의 이름은 무엇입니까?"
)

messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 제이슨 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

In [ ]:
# 주로 이런 형식으로 사용
llm.invoke(messages).content

'제 이름은 제이슨입니다. 당신은 어떻게 부르길 원하시나요?'

### LangChain

- 여러 작업을 순차적으로 연결해서(Chain) 복잡한 워크플로우를 구축

In [17]:
chain = template | llm

chain.invoke({
    "name": "Jason", 
    "user_input": "당신의 이름은 무엇입니까?"
}).content

'제 이름은 Jason입니다. 당신은 어떻게 부르길 원하시나요?'

### LECL 인터페이스

- `Runnable` 클래스 (사용자 정의 체인을 쉽게 만들 수 있게 하는 프로토콜)

> `stream`: 응답의 청크를 스트리밍 (실시간 출력)  
> `invoke`: 입력에 대해 체인을 호출 (호출)  
> `batch`: 입력 목록에 대해 체인을 호출 (배치단위 호출)

- 비동기 방식도 존재
> `astream`, `ainvoke`, `abatch`, `astream_log`

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI()

# 주어진 토픽에 대한 요약을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")

chain = prompt | model | StrOutputParser()

In [19]:
result = chain.invoke({
    'topic': 'ChatGPT'
})

result

'ChatGPT는 OpenAI에서 개발한 대화형 인공지능 모델로, 다양한 주제에 대한 대화 및 질문에 응답할 수 있습니다. 이 모델은 기존에 존재하는 데이터를 학습하여 텍스트 생성 및 이해 능력을 향상시켰습니다. ChatGPT를 활용하면 대화형 서비스나 개인 비서 등 다양한 분야에서 활용할 수 있습니다.'

In [21]:
results = chain.batch([
    {'topic': 'ChatGPT'},
    {'topic': 'Instagram'},
])

results

[' ChatGPT는 대화형 인공지능 프로그램으로, 사용자와 대화를 나누며 다양한 주제에 대해 대화할 수 있습니다. ChatGPT는 자연어 처리 기술을 기반으로 작동하여, 사용자의 질문에 응답하거나 소셜 봇으로 작동할 수 있습니다. ChatGPT는 온라인 상에서 커뮤니케이션 및 정보 교환을 하는 데 도움을 줄 수 있는 유용한 도구입니다.',
 '인스타그램은 사진과 동영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 다양한 필터와 효과를 통해 자신의 콘텐츠를 꾸밀 수 있다. 또한 해시태그를 활용하여 관심사나 주제에 맞는 게시물을 찾아볼 수 있고, 팔로워와 소통하며 커뮤니티를 형성할 수 있다. 인플루언서나 브랜드들 또한 인스타그램을 활용하여 마케팅과 홍보에 활발하게 이용하고 있다.']

In [22]:
chain.batch([
    {'topic': 'ChatGPT'},
    {'topic': 'Instagram'},
    {'topic': '멀티모달'},
    {'topic': '프로그래밍'},
    {'topic': '머신러닝'},
], config={'max_concurrency': 3}) # 동시 처리 작업 수 지정

['ChatGPT는 혁신적인 자연어 처리 기술을 사용하여 대화 상대와 자연스럽게 대화할 수 있는 AI 챗봇 플랫폼이다. 이 기술은 사용자의 질문에 대답하고 다양한 주제에 대해 대화할 수 있는 지능적이고 학습능력이 있는 모델을 제공한다. ChatGPT를 통해 사람과의 상호작용을 최적화하고 다양한 상황에서 유용한 정보나 서비스를 제공할 수 있다.',
 'Instagram은 사진과 동영상을 공유하고 소셜 네트워크 서비스를 이용할 수 있는 모바일 앱이다. 사용자들은 팔로워와 소통하고 다양한 콘텐츠를 즐기며 일상을 공유할 수 있다. 인기 있는 인플루언서나 브랜드들도 활발히 활동하는 플랫폼이다.',
 '멀티모달은 여러 가지 다른 형식의 정보를 동시에 사용하여 효율적으로 커뮤니케이션하는 방법이다. 이는 텍스트, 이미지, 음성, 비디오 등 다양한 매체를 결합하여 정보를 전달하고 이해하는 데 도움을 준다. 멀티모달을 활용하면 더 다양한 사용자들에게 접근할 수 있고, 효과적인 소통과 학습을 도모할 수 있다.',
 '프로그래밍은 컴퓨터에게 원하는 작업을 수행하도록 지시하는 과정을 말합니다. 이를 위해 프로그래머는 특정 프로그래밍 언어를 사용하여 코드를 작성하고 디버깅하는 작업을 합니다. 프로그래밍을 통해 다양한 소프트웨어나 애플리케이션을 개발할 수 있으며, 현대 사회에서는 필수적인 기술로 인정받고 있습니다.',
 '머신러닝은 컴퓨터 시스템이 데이터를 사용해 패턴을 학습하고 예측하는 기술입니다. 이를 통해 프로그램이 스스로 데이터로부터 지식을 습득하고 문제를 해결할 수 있게 됩니다. 머신러닝은 효율적이고 정확한 결정을 내릴 수 있는 스마트한 시스템을 만들기 위해 활발히 연구되고 활용되고 있습니다.']

### RunnableParallel

- 여러 체인을 병렬 처리

In [ ]:
from langchain_core.runnables import RunnableParallel

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | model
    | StrOutputParser()
)

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?") # 변수가 같을 필요는 없음
    | model
    | StrOutputParser()
)

# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)

In [ ]:
combined.invoke({"country": "대한민국"})
#combined.invoke({"country": "대한민국", "country2": "미국"})

{'capital': '대한민국의 수도는 서울입니다.', 'area': '대한민국의 총 면적은 약 100,363km² 입니다.'}

### Output Parsers (출력 파서)

- 출력을 원하는 구조화된 형태로 변환

#### PydanticOutputParser
- `pydantic` 모듈의 `BaseModel`과 `Field`를 활용하여 객체의 구조를 정의

In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [26]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [27]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

# 출력 형태를 프롬프트에 추가한다
"""  
다음의 출력형식으로 답변해줘
person: str (메일을 보낸 사람)
...
"""

# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [28]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [29]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

In [ ]:
# chain 을 생성합니다.
chain = prompt | llm 

In [32]:
# chain 을 실행하고 결과를 출력합니다.
output = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
print(output.content)

```json
{
    "person": "김철수",
    "email": "chulsoo.kim@bikecorporation.me",
    "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
    "summary": "김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 브로슈어를 요청하고, 기술 사양, 배터리 성능, 디자인 정보를 필요로 합니다. 또한, 협력 논의를 위해 1월 15일 화요일 오전 10시에 미팅을 제안합니다.",
    "date": "1월 15일 화요일 오전 10시"
}
```


In [33]:
structured_output = parser.parse(output)
print(structured_output)

ValidationError: 1 validation error for Generation
text
  Input should be a valid string [type=string_type, input_value=AIMessage(content='```jso...o': 0, 'reasoning': 0}}), input_type=AIMessage]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [ ]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.(이런 형태임)
chain = prompt | llm | parser

In [35]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="김철수 상무는 바이크코퍼레이션의 자전거 제조 및 유통 경험을 바탕으로, 테디인터내셔널의 신규 자전거 'ZENESIS'에 대한 유통 협력을 제안하며, 기술 사양, 배터리 성능, 디자인 정보가 포함된 브로슈어를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안합니다.", date='1월 15일 화요일 오전 10시')

In [36]:
print(response.person)
print(response.email)

김철수
chulsoo.kim@bikecorporation.me


### JsonOutputParser

- Json 스키마를 지정하여 결과를 JSON 포맷으로 결과를 전달합니다.
- 모델의 파라미터 수, 즉 모델이 충분히 커야 정확하게 생성할 수 있습니다.

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [38]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

In [39]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [ ]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser  # 체인을 구성합니다.

chain.invoke({"question": question})  # 체인을 호출하여 쿼리 실행

{'description': '지구 온난화는 지구 평균 기온이 상승하여 극심한 기후 변화, 해수면 상승, 생태계 파괴 등 심각한 환경 문제를 초래합니다.',
 'hashtags': '#지구온난화 #기후변화 #환경보호 #지구살리기'}

In [41]:
# 문자열이 아니라 
result = chain.invoke({"question": question})
result['hashtags']

'#지구온난화 #기후변화 #환경보호 #지구살리기'

In [42]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "주제에 대한 간결한 설명", "title": "Description", "type": "string"}, "hashtags": {"description": "해시태그 형식의 키워드(2개 이상)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [ ]:
# Pydantic 없이 사용가능

In [ ]:
# 질의 작성
question = """ \
"지구 온난화에 대해 알려주세요. " \
"온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요." \
"""

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
        # Format: {format_instructions}\n\n
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)

{'description': '지구 온난화는 대기 중 온실가스 농도의 증가로 인해 지구 평균 기온이 상승하는 현상입니다. 이는 극지방의 빙하 감소, 해수면 상승, 기상이변 증가 등 다양한 환경 변화를 초래합니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경보호', '#지구환경']}


#### 기타 출력파서

> CommaSeparatedListOutputParser  
> StructuredOuputParser  
> PandasDataFrameOutputParser  
> DatetimeOutputParser  
> EnumOutputParser


In [ ]:
# ENUM (상수값들 모임)

class Category:
    FLOWER: str
    FOOD: str

# pd.DataFrame({
#     'key': [values]
# })

### 캐싱 (Caching)

- `동일한 작업`을 여러 번 수행해야 할 경우 API 호출 횟수를 줄이고 응답 속도를 빠르게 할 수 있음

#### InMemoryCache

In [44]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 모델을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm

In [ ]:
%%time # 실행시간 측정
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 나라로 경제적으로 선진화된 나라이면서도 전통과 현대가 공존하는 다양한 문화를 지니고 있다. 서울을 중심으로 현대적인 도시들이 발달하고 있으며 한류 열풍으로 한국 문화가 전 세계에 널리 알려져 있다. 또한 한반도 분단으로 인해 북한과의 관계가 여전히 긴장 상태에 있는 것이 한국의 중요한 이슈 중 하나이다. 미래 지향적인 기술 발전과 함께 전통 문화와 역사를 소중히 여기는 한국은 매력적인 나라로 손꼽힌다.
CPU times: total: 125 ms
Wall time: 3.45 s


In [ ]:
# %pip install -U langchain-community

In [ ]:
%%time

from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

In [ ]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

### SQLiteCache

In [38]:
import os
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [ ]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

### 메모리 (Memory)

- 대화 기록을 저장하는 방법 (history)

### ConversationBufferMemory

In [54]:
# Chain 추가하는 방법
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI


# ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI()

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [ ]:
memory.load_memory_variables({})  # 메모리 변수를 빈 딕셔너리로 초기화합니다.

In [57]:
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
)

In [ ]:
runnable.invoke({"input": "hi"})

In [59]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [60]:
chain = runnable | prompt | model

In [ ]:
# chain 객체의 invoke 메서드를 사용하여 입력에 대한 응답을 생성합니다.
response = chain.invoke({"input": "만나서 반갑습니다. 제 이름은 제이슨입니다."})
print(response.content)  # 생성된 응답을 출력합니다.

In [ ]:
memory.load_memory_variables({})

In [ ]:
memory.save_context(
    {"human": "만나서 반갑습니다. 제 이름은 제이슨입니다."}, {"ai": response.content}
)

memory.load_memory_variables({})

In [ ]:
# 이름을 기억하고 있는지 추가 질의합니다.
response = chain.invoke({"input": "제 이름이 무엇이었는지 기억하세요?"})
# 답변을 출력합니다.
print(response.content)

#### 대화 내용을 자동으로 저장하는 커스텀 메모리 관리

In [65]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [66]:
class MyConversationChain(Runnable):

    def __init__(self, llm, prompt, memory, input_key="input"):

        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key

        self.chain = (
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(self.memory.load_memory_variables)
                | itemgetter(memory.memory_key)  # memory_key 와 동일하게 입력합니다.
            )
            | prompt
            | llm
            | StrOutputParser()
        )

    def invoke(self, query, configs=None, **kwargs):
        answer = self.chain.invoke({self.input_key: query})
        self.memory.save_context(inputs={"human": query}, outputs={"ai": answer})
        return answer


#### 메모리 종류

- ConversationBufferMemory(대화 버퍼 메모리): 메시지를 저장한 다음 변수에 메시지를 추출
- ConversationBufferWindowMemory(대화 버퍼 윈도우 메모리): 시간이 지남에 따라 k개만큼만 사용
- ConversationTokenBufferMemory(대화 토큰 버퍼 메모리): 최근 대화 저장 시 토큰 길이를 사용
- ConversationEntityMemory(대화 엔티티 메모리): 특정 엔티티에 대한 주어진 사실을 기억
- ConversationKGMemory(대화 지식그래프 메모리): 지식 그래프를 활용해 서로 다른 개체 간 관계를 이해
- ConversationSummaryMemory(대화 요약 메모리): 대화의 요약을 생성하여 기억
- VectorStoreRetrieverMemory(벡터저장소 검색 메모리): 벡터 스토어에서 가장 눈에 띄는 상위 K개 문서 쿼리

In [70]:
# ChatOpenAI 모델을 초기화합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# 요약 메모리로 교체할 경우
# memory = ConversationSummaryMemory(
#     llm=llm, return_messages=True, memory_key="chat_history"
# )

conversation_chain = MyConversationChain(llm, prompt, memory)

In [ ]:
conversation_chain.invoke("안녕하세요? 만나서 반갑습니다. 제 이름은 제이슨 입니다.")

In [ ]:
conversation_chain.invoke("제 이름이 뭐라고요?")

### SQLite에 대화내용 저장

In [78]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 파일을 설정
chat_message_history = SQLChatMessageHistory(
    session_id="sql_history", connection="sqlite:///sqlite.db"
)

In [79]:
# 사용자 메시지를 추가합니다.
chat_message_history.add_user_message(
    "안녕? 만나서 반가워. 내 이름은 제이슨이야. 나는 랭체인 개발자야. 앞으로 잘 부탁해!"
)
# AI 메시지를 추가합니다.
chat_message_history.add_ai_message("안녕 제이슨, 만나서 반가워. 나도 잘 부탁해!")

In [ ]:
# 채팅 메시지 기록의 메시지들
chat_message_history.messages

In [81]:
# Chain에 적용

In [82]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [83]:
prompt = ChatPromptTemplate.from_messages(
    [
        # 시스템 메시지
        ("system", "You are a helpful assistant."),
        # 대화 기록을 위한 Placeholder
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),  # 질문
    ]
)

# chain 을 생성합니다.
chain = prompt | ChatOpenAI(model_name="gpt-4o") | StrOutputParser()

In [84]:
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db",
    )

In [85]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True,
    ),
]

In [86]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,  # 대화 기록을 가져오는 함수를 설정합니다.
    input_messages_key="question",  # 입력 메시지의 키를 "question"으로 설정
    history_messages_key="chat_history",  # 대화 기록 메시지의 키를 "history"로 설정
    history_factory_config=config_fields,  # 대화 기록 조회시 참고할 파라미터를 설정합니다.
)

In [87]:
# config 설정
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

In [ ]:
# 질문과 config 를 전달하여 실행합니다.
chain_with_history.invoke({"question": "안녕 반가워, 내 이름은 제이슨이야"}, config)

In [ ]:
# 후속 질문을 실해합니다.
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)

In [ ]:
# config 설정
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation2"}}

# 질문과 config 를 전달하여 실행합니다.
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)